<a href="https://colab.research.google.com/github/Slimani-CE/fake-news-classifier/blob/main/Fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link to dataset: https://www.kaggle.com/c/fake-news

bout the Dataset:
  1. id: Unique id for a news article
  2. title: The title of a news article
  3. author: Author of the news article
  4. text: The text of the article; could be incomplete
  5. label: A label that marks whether the news article is real or fake:

    0.   Fake news
    1.   Real news

## Importing libraries

In [1]:
import numpy as pn
import pandas as pd
import re
from nltk.corpus import stopwords # Remove stop words from our dataset
from nltk.stem.porter import PorterStemmer # Give the root word of our dataset
from sklearn.feature_extraction.text import TfidfVectorizer # Convert our dataset into vectors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# English stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Import the dataset

In [4]:
dataset = pd.read_csv('drive/MyDrive/Classroom/31 Projects/Project 4: Fake News Prediction With Python/train.csv')

## Data Pre-processing

In [5]:
dataset.shape

(20800, 5)

In [6]:
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
def display_missing_values(dataset):
  na_df = dataset.isna().sum().to_frame()
  na_df.columns = ['NA Count']
  na_df = na_df[na_df['NA Count'] != 0]
  return na_df
display_missing_values(dataset)

,NA Count
title,558
author,1957
text,39


In [8]:
# Replace the null values with empty string
dataset = dataset.fillna('')

In [10]:
# Merging the title and the author name
dataset['content'] = dataset['author'] + ' ' + dataset['title'] + ' ' + dataset['text']

In [11]:
dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [12]:
port_stem = PorterStemmer()
new_stopwords = stopwords.words('english')
new_stopwords.remove('not')

In [13]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in new_stopwords]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [14]:
dataset['content'] = dataset['content'].apply(stemming)

In [15]:
dataset[['content']]

,content
0,darrel lucu hous dem aid even see comey letter...
1,daniel j flynn flynn hillari clinton big woman...
2,consortiumnew com truth might get fire truth m...
3,jessica purkiss civilian kill singl us airstri...
4,howard portnoy iranian woman jail fiction unpu...
...,...
20795,jerom hudson rapper trump poster child white s...
20796,benjamin hoffman n f l playoff schedul matchup...
20797,michael j de la merc rachel abram maci said re...
20798,alex ansari nato russia hold parallel exercis ...


In [16]:
# Separating the data & label
X = dataset['content'].values
y = dataset['label'].values

In [17]:
# Vectorize the textual data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

## Split dataset to training and test sets

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2, random_state = 42)

## Training the Logistic Regression model

In [19]:
model = LogisticRegression()

In [20]:
model.fit(X_train, y_train)

LogisticRegression()

## Evaluation

In [21]:
# Accuracy score on training data
train_prediction = model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_prediction)
print("Accuracy score on training data: ", train_accuracy)

Accuracy score on training data:  0.978125


In [22]:
# Accuracy score on test data
test_prediction = model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_prediction)
print("Accuracy score on test data: ", test_accuracy)

Accuracy score on test data:  0.9576923076923077


## Making a predictive System

In [23]:
X_new = X_test[0]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real
